In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymysql

In [ ]:
url='https://movie.naver.com/movie/running/current.naver'
base='https://movie.naver.com/'
res=requests.get(url).content
soup=bs(res,'html.parser')
#print(soup)
movies=soup.select('div.lst_wrap > ul > li') # 해당하는 선택자의 li태그들을 리스트형태로 저장
#print(movies[0].text)
movie_list=[]
for movie in movies:
    title=movie.select('dl > dt > a')[0].text # 해당 선택자를 리스트형태로 저장하여 첫번째에 해당하는 텍스트를 출력
    reated=movie.select_one('dl > dt > span').text
    point=movie.select_one('dl > dd.star > dl > dd > div > a > span.num').text
    #print(title,reated,point)
    info=movie.select_one('dl.info_txt1 > dd').text
    #arr_info=info.split('|')
    #print(arr_info)
    arr_info=info.replace('\t','').replace('\n\n','').replace('\r\n','').split('|')
    if len(arr_info)<3:
        arr_info.insert(0,'장르정보 없음')
    genre=arr_info[0]
    running=arr_info[1].replace('분','')
    release_date=arr_info[2].replace('개봉','')
    #print(genre,running,release_date)
    director=movie.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(4) > span > a').text
    #print(director)
    actors=movie.select('dl > dd:nth-child(3) > dl > dd:nth-child(6) > span > a')
    #print(actors)
    actorstr=''
    for actor in actors:
        actorstr+=actor.text+','
    actors=actorstr.strip(',')
    #movie_list.append((title,reated,point,genre,running,release_date,director,actorstr))
    # 상세페이지 
    href=movie.select('dl > dt > a')[0]['href']
    url2=base+href
    res2=requests.get(url2)
    soup2=bs(res2.text,'html.parser')
    story=soup2.select('#content > div.article > div.section_group.section_group_frst > div:nth-child(1) > div > div.story_area > p')[0].text.replace('\r','')
    #print(story)
    
    temp=soup2.select('#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph')
    if len(temp)>0:
        temp=temp[0].text.replace('이상','').replace('\n','').split('대')
        temp=temp[:-1]
        temp=' '.join(temp)
    else:
        temp='선호도 정보 없음'
    #print(temp)
    movie_list.append((title,reated,point,genre,running,release_date,director,actorstr,story,temp))

In [ ]:
df=pd.DataFrame(movie_list,columns=['title','rated','point','genre','running','relase_date','director','actors','stroy','temp'])
df

In [ ]:
conn=pymysql.connect(host='localhost',user='jhs',password='1234',db='pydb',charset='utf8')
cursor=conn.cursor()
sql="insert into movie_list(title,rated,point,genre,running,relase_date,director,actors,story,temp) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql,movie_list)
conn.commit()
conn.close()